<p align="center">
    <img width="40%" src="https://upload.wikimedia.org/wikipedia/commons/1/16/Fanpage.it.png">
</p>
Dataset per Abstract Summarization

In [ ]:
import requests

# pagina
r = requests.get("https://www.fanpage.it/politica/")

In [ ]:
!pip install -q colorama

In [ ]:
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
import colorama

 colori
colorama.init()
GREEN = colorama.Fore.GREEN
GRAY = colorama.Fore.LIGHTBLACK_EX
RESET = colorama.Fore.RESET
YELLOW = colorama.Fore.YELLOW

internal_urls = set()
external_urls = set()

def is_valid(url):
    """
    Controlla che l'url sia valido.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

In [ ]:
def get_all_website_links(url):
    """
    Tutti gli urls che vengono trovati nell'url che appartiene al sito
    """
    # tutti gli URLs `url`
    urls = set()
    # dominio senza prot
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
      href = a_tag.attrs.get("href")
      if href == "" or href is None:
          # href tag vuota
          continue
      href = urljoin(url, href)
      parsed_href = urlparse(href)
      # rimuovi parametri URL GET, URL fragments, etc.
      href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path

      if not is_valid(href):
          # URL non valido
          continue
      if href in internal_urls:
          # già visto
          continue
      if domain_name not in href:
          # link estrerno
          #if href not in external_urls:
              #external_urls.add(href)
          continue
      urls.add(href)
      internal_urls.add(href)
    return urls

In [ ]:
# numero di url visitati
total_urls_visited = 0

def crawl(url, max_urls=50):
    """
    Crawlerrr
    `external_urls` e `internal_urls` global set variables
    """
    global total_urls_visited
    total_urls_visited += 1
    if (total_urls_visited%500 == 0):
      print("Total URLs visited: ", total_urls_visited)
      print(f"{YELLOW}[*] Now Crawling: {url}{RESET}")
    
    links = get_all_website_links(url)
    links = [link for link in links if link in internal_urls]
    for link in links:
        if total_urls_visited > 3000:
            break
        try:
          crawl(link, max_urls=max_urls)
        except: 
          pass

In [ ]:
import sys
sys.setrecursionlimit(10000)

In [ ]:
# setting internal_urls-external_urls empty
internal_urls = set()
external_urls = set()

crawl("https://www.fanpage.it/")

In [ ]:
len(internal_urls)

In [ ]:
# save text file with urls
textfile = open("fanpage_story.txt", "w")

for element in internal_urls:
    textfile.write(element + "\n")
textfile.close()

# Dataset

In [ ]:
!pip install -q --upgrade spacy
!pip install -q rouge_score

In [ ]:
import spacy.cli
import string

spacy.cli.download("it_core_news_sm")
nlp = spacy.load("it_core_news_sm")

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# split summary into sentences and compute rougeL with article text
def rouge_l(titolo, testo): 
  testo = testo.translate(str.maketrans('', '', string.punctuation))
  doc = nlp(titolo)
  for sents in doc.sents:
    sents = str(sents)
    sents = sents.translate(str.maketrans('', '', string.punctuation))
    if scorer.score(testo, sents).get('rougeL')[0] >= 0.95:
      return False
  return True

In [ ]:
from tqdm import tqdm
titles = []
articles = []

# get abstract and article from fanpage urls
num_lines = sum(1 for line in open("fanpage_complete.txt",'r'))
with open("fanpage_complete.txt",'r') as f:
    for line in tqdm(f, total=num_lines):
      try:
        url = str(line).rstrip()
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')        
        titolo_articolo = soup.find("div", {"class":"fp_intro__abstract"}).get_text()
        try:
          testo_articolo = soup.find_all("p")[1:10]
        except:
          testo_articolo = ""
          
        paragraphtext = []
        for paragraph in testo_articolo[:-1]:
            # testo
            text = paragraph.get_text()
            paragraphtext.append(text)

        testo_fin = ' '.join(paragraphtext) 
        # controllo intersezione e che il titono non sia nell'articolo
        if rouge_l(titolo_articolo,testo_fin):
          titles.append(titolo_articolo)
          articles.append(testo_fin)
      except:
        pass

In [ ]:
import pandas as pd
import numpy as np

# some cleaning removing empty rows 
df = pd.DataFrame({"source":articles, "target":titles})
df.head()
df.shape[0]

df['source'].replace("", np.nan, inplace=True)
df['target'].replace("", np.nan, inplace=True)
df.dropna(subset=['source'], inplace=True)
df.dropna(subset=['target'], inplace=True)
df = df.reset_index(drop=True)

print(df.source.size, df.target.size)

In [ ]:
df.to_csv("/content/Fanpage_design.csv", sep=',', index=False)

### plots

In [ ]:
import re
word_bound = re.compile(r'\b')

def num_words(line):
    return len(word_bound.findall(line)) >> 1

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') 
from matplotlib.colors import to_rgba

In [ ]:
ii = [i for i in range(0,80)]
lunghezze_riass = [num_words(df["target"].iloc[i]) for i in range(df.shape[0])]

fig = plt.figure()
ax = fig.add_subplot()
fig.subplots_adjust(top=0.85)

fig.suptitle('lunghezze riassunti', fontsize=14, fontweight='bold')
ax.set_title('Fanpage')

ax.set_xlabel('tokens')
ax.set_ylabel('numero')

#plt.style.use(['dark_background'])
plt.hist(lunghezze_riass, bins=ii, color='c')

plt.show()

In [ ]:
ii = [i for i in range(0,500,6)]
lunghezze_docu = [num_words(df['source'].iloc[i]) for i in range(df.shape[0])]

fig = plt.figure()
ax = fig.add_subplot()
fig.subplots_adjust(top=0.85)

fig.suptitle('lunghezze documenti', fontsize=14, fontweight='bold')
ax.set_title('Fanpage')

ax.set_xlabel('tokens')
ax.set_ylabel('numero')

#plt.style.use(['dark_background'])
plt.hist(lunghezze_docu, bins=ii, color='c')
plt.show()

# text cleaning

We used textacy for text cleaning, normalizing unicode, removing html tags and normalizing whitespaces 

In [ ]:
!pip install -q textacy 

In [ ]:
import pandas as pd

# read dataset csv
df = pd.read_csv("/content/Fanpage.csv")

In [ ]:
from textacy import preprocessing
clean_inputs = []
clean_targets = []

preproc = preprocessing.make_pipeline(
     preprocessing.normalize.unicode,
     preprocessing.remove.html_tags,
     preprocessing.normalize.whitespace
)

# apply to dataset elements
clean_inputs = [preproc(df["source"].iloc[i]) for i in range(df.shape[0])]
clean_targets = [preproc(df["target"].iloc[i]) for i in range(df.shape[0])]
df = pd.DataFrame({"source": clean_inputs, "target": clean_targets})

In [ ]:
df.to_csv("/content/Fanpage_clean.csv", sep=',', index=False)

# sets for filtering 

In [ ]:
from os import listdir
from os.path import isfile, join

def get_lines(file):
  return open(file, 'r').readlines()

mypath = "/content/total/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

total = 0
filter = set()
for file in onlyfiles:
  lines = get_lines(mypath + file)
  total += len(lines)
  filter.update(lines)

print(total, len(filter))

lines = [line.rstrip() for line in filter]

In [ ]:
textfile = open("fanpage_complete.txt", "w")

for element in lines:
    textfile.write(element + "\n")
textfile.close()